In [386]:
import movingpandas as mpd
import geopandas as gp
import pandas as pd
from datetime import timedelta
from shapely import LineString, Point
import skmob
from skmob.preprocessing import detection
import numpy as np

In [3]:
# read geolife pickle file
geolife_raw_gdf = gp.GeoDataFrame(pd.read_pickle('data/geolife/geolife_raw.pkl'))


In [423]:
traj_collection = mpd.TrajectoryCollection(geolife_raw_gdf, 'user', t='time')

In [424]:
my_traj = traj_collection.trajectories[1]
print(my_traj)

Trajectory 1 (2008-10-23 05:53:05 to 2008-12-15 00:31:18) | Size: 108607 | Length: 1018053.3m
Bounds: (116.145054, 39.900944, 116.443578, 40.076116)
LINESTRING (116.319236 39.984094, 116.319322 39.984198, 116.319402 39.984224, 116.319389 39.984211, 


In [425]:
split = mpd.StopSplitter(my_traj).split(max_diameter=100, min_duration=timedelta(minutes=15), min_length=200)
split

TrajectoryCollection with 108 trajectories

In [426]:
split.to_traj_gdf()

,traj_id,start_t,end_t,geometry,length,direction
0,1_2008-10-23 05:53:05,2008-10-23 05:53:05,2008-10-23 11:10:29,"LINESTRING (116.31924 39.98409, 116.31932 39.9...",11150.474980,341.716224
1,1_2008-10-23 23:44:06,2008-10-23 23:44:06,2008-10-24 00:23:18,"LINESTRING (116.30590 40.01412, 116.30579 40.0...",6389.233020,156.261271
2,1_2008-10-24 01:46:57,2008-10-24 01:46:57,2008-10-24 02:03:11,"LINESTRING (116.32688 39.97878, 116.32699 39.9...",2140.426179,279.035617
3,1_2008-10-24 02:29:29,2008-10-24 02:29:29,2008-10-24 03:27:05,"LINESTRING (116.30847 39.98105, 116.30864 39.9...",1166.429475,111.959781
4,1_2008-10-24 03:50:05,2008-10-24 03:50:05,2008-10-24 04:13:35,"LINESTRING (116.31410 39.97956, 116.31424 39.9...",2073.172740,99.278008
...,...,...,...,...,...,...
103,1_2008-12-13 12:04:01,2008-12-13 12:04:01,2008-12-13 12:36:48,"LINESTRING (116.34341 39.97828, 116.34344 39.9...",1415.976143,129.111053
104,1_2008-12-13 12:54:15,2008-12-13 12:54:15,2008-12-13 13:44:33,"LINESTRING (116.34469 39.97805, 116.34481 39.9...",8441.780588,324.995328
105,1_2008-12-13 14:01:47,2008-12-13 14:01:47,2008-12-13 14:17:31,"LINESTRING (116.31134 40.01429, 116.31125 40.0...",1397.923951,262.797678
106,1_2008-12-14 08:08:53,2008-12-14 08:08:53,2008-12-14 08:46:01,"LINESTRING (116.30599 40.01411, 116.30586 40.0...",6165.764823,155.864986
